# Max Flow LP (Manhattan)

### Author: [Ben Rosenberg](https://benrosenberg.info)

In [13]:
from ortools.linear_solver import pywraplp as OR
import time
import pandas as pd

In [14]:
df_manhattan_graph = pd.read_csv('manhattan_street_capacities.csv')
print(df_manhattan_graph.head())

   source_osm  target_osm        street_name  capacity      u_lat      u_lon  \
0    42421728    42435337  Central Park West      1000  40.798048 -73.960044   
1    42421728    42421731  West 106th Street      1000  40.798048 -73.960044   
2    42421728    42432736  Central Park West      1000  40.798048 -73.960044   
3    42421731    42437916   Manhattan Avenue       200  40.798654 -73.961474   
4    42421731    42432737   Manhattan Avenue       200  40.798654 -73.961474   

       v_lat      v_lon  
0  40.797376 -73.960535  
1  40.798654 -73.961474  
2  40.798726 -73.959547  
3  40.797976 -73.961970  
4  40.799330 -73.960978  


In [15]:
nodes = list(df_manhattan_graph['source_osm'].unique())
# choose source to be the world trade center, and choose sink to be Hunter College
source, sink = 42453395, 42431678

edges = list(zip(df_manhattan_graph['source_osm'], df_manhattan_graph['target_osm']))

# capacity[i,j] is the capacity of edge (i,j)
# Use groupby to sum capacities if there are multiple edges between the same nodes
capacity = df_manhattan_graph.groupby(['source_osm', 'target_osm'])['capacity'].sum().to_dict()

In [16]:
start_time = time.time()

m = OR.Solver('Max Flow', OR.Solver.GLOP_LINEAR_PROGRAMMING)

# add variable f
f = {}
for (i,j) in edges:
    f[i,j] = m.NumVar(0, m.infinity(), 'f[{},{}]'.format(
        i,j
    ))

# add constraint flow in/out for non-source/sink nodes
for i in nodes:
    if i not in (nodes[0], nodes[-1]):
        m.Add(sum(f[j,i] for (j,x) in edges if x == i) - 
              sum(f[i,j] for (x,j) in edges if x == i)
              == 0)

# add constraint on edge flows w.r.t. capacities
for (i,j) in edges:
    m.Add(f[i,j] <= capacity[i,j])

# add constraint on edge flows w.r.t. 0
for (i,j) in edges:
    m.Add(f[i,j] >= 0)
    
# set objective
m.Maximize(sum(f[i,j] for (i,j) in edges if j == sink))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 7 seconds]
Objective: 1200.0


In [17]:
# optimal solution
for (i,j) in edges[:10]:
    print('f[{},{}] = {}'.format(
        i, j, f[i,j].solution_value()
    ))

f[42421728,42435337] = 800.0000000000003
f[42421728,42421731] = 400.0
f[42421728,42432736] = 1000.0
f[42421731,42437916] = 200.0
f[42421731,42432737] = 0.0
f[42421731,42421728] = 0.0
f[42421731,42421737] = 600.0
f[42421737,42421731] = 0.0
f[42421737,42437917] = 0.0
f[42421737,42421741] = 600.0


In [18]:
import plotly.graph_objects as go

# 1. Prepare lists for the "One-Trace" trick
lats = []
lons = []
hover_text = []

# 1. Extract the flow values into a list
flow_data = []
for (i, j) in edges:
    flow_data.append({
        'source_osm': i,
        'target_osm': j,
        'flow_result': f[i, j].solution_value()
    })

# 2. Convert to a DataFrame
df_flows = pd.DataFrame(flow_data)

# 3. Merge with your existing geography data (if you have it)
# This adds the street names, lat/long, and capacities to your results
results_df = pd.merge(
    df_manhattan_graph, 
    df_flows, 
    on=['source_osm', 'target_osm'], 
    how='left'
)

# 4. Calculate Utilization (for the heatmap)
# results_df['utilization'] = results_df['flow_result'] / results_df['capacity']
results_df['utilization'] = results_df['flow_result'] / results_df['flow_result'].max()

# Peek at the top bottlenecks
print(results_df.sort_values(by='utilization', ascending=False).head())

color_bins = [
    ('rgb(200, 200, 200)', -0.01, 0.00, 'No Flow'),      # Gray
    ('rgb(0, 255, 0)', 0.00, 0.25, 'Low (0-25%)'),      # Green
    ('rgb(255, 255, 0)', 0.25, 0.50, 'Med (25-50%)'),    # Yellow
    ('rgb(255, 165, 0)', 0.50, 0.75, 'High (50-75%)'),   # Orange
    ('rgb(255, 0, 0)', 0.75, 1.01, 'Critical (75-100%)') # Red
]

fig = go.Figure()

# 2. Iterate through bins and add one trace per bin
for color, lower, upper, label in color_bins:
    # Filter the DF for this specific utilization range
    bin_df = results_df[(results_df['utilization'] > lower) & (results_df['utilization'] <= upper)]
    
    if bin_df.empty:
        continue
        
    lats, lons = [], []
    for row in bin_df.itertuples():
        lats.extend([row.u_lat, row.v_lat, None])
        lons.extend([row.u_lon, row.v_lon, None])
    
    # Add this bin as a single trace
    fig.add_trace(go.Scattermapbox(
        lat=lats,
        lon=lons,
        mode='lines',
        line=dict(width=2 if lower < 0 else 3 + (lower * 5), color=color),
        name=label,
        hoverinfo='none' # Hover is hard on 4k lines; keeping it clean
    ))

# Extract coordinates from the node attributes in G
get_lat = lambda node : df_manhattan_graph[df_manhattan_graph['source_osm'] == node].iloc[0]['u_lat']
get_lon = lambda node : df_manhattan_graph[df_manhattan_graph['source_osm'] == node].iloc[0]['u_lon']
src_lat = get_lat(source)
src_lon = get_lon(source)
snk_lat = get_lat(sink)
snk_lon = get_lon(sink)

# 2. Add the trace with Bold HTML tags
# Add the trace using texttemplate for bold styling
fig.add_trace(go.Scattermapbox(
    lat=[src_lat, snk_lat],
    lon=[src_lon, snk_lon],
    mode='markers+text',
    marker=dict(size=18, color=["#00FFFF", '#00FFFF']), 
    
    textposition="top right",
    textfont=dict(
        size=16,
        color="cyan" 
    ),
    name='Endpoints'
))

# Optional: Re-center the map exactly between the two points
fig.update_layout(
    mapbox=dict(
        center=dict(lat=(src_lat + snk_lat) / 2, lon=(src_lon + snk_lon) / 2),
        zoom=13
    )
)

# 4. Final Layout
fig.update_layout(
    mapbox=dict(
        style="carto-darkmatter", # Dark mode makes the colors pop
        zoom=11
    ),
    margin={"r":0,"t":40,"l":0,"b":0},
    title="Manhattan (WTC->Hunter) Traffic Flow",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(0,0,0,0.5)", font=dict(color="white"))
)

fig.write_html("manhattan_traffic.html")
print('Plot written to `manhattan_traffic.html`.')

      source_osm  target_osm                 street_name  capacity      u_lat  \
8866  4142105813    42440838                Canal Street      3500  40.723189   
173     42423565  3785586748  Queensboro Bridge Approach      3500  40.759643   
410     42427316    42440810                Canal Street      3500  40.718019   
131     42422399    42458435        Henry Hudson Parkway      3500  40.788003   
9659  9902723823  9902723825              Unnamed Street      3500  40.797050   

          u_lon      v_lat      v_lon  flow_result  utilization  
8866 -74.007217  40.722569 -74.006317       3500.0          1.0  
173  -73.964000  40.760013 -73.963753       3500.0          1.0  
410  -73.999953  40.717554 -73.999309       3500.0          1.0  
131  -73.982759  40.795685 -73.976732       3500.0          1.0  
9659 -73.920620  40.796622 -73.919733       3500.0          1.0  
Plot written to `manhattan_traffic.html`.


C:\Users\xethr\AppData\Local\Temp\ipykernel_17304\2798742592.py:60: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\xethr\AppData\Local\Temp\ipykernel_17304\2798742592.py:79: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

